In [0]:
USE data_science.default

In [0]:
CREATE OR REPLACE TABLE lw_20250903_kpi_table_by_j as
with growth as (
  select
  app_package,
  country_group,
  mediasource,
  tag,
  install_day,
  p_r14_r7 * p_r30_r14 * p_r60_r30 * p_r90_r60 * p_r120_r90 as p_r120_r7,
  p_r14_r7 * p_r30_r14 * p_r60_r30 * p_r90_r60 * p_r120_r90 * p_r135_r120 as p_r135_r7
  from lw_20250903_predict_revenue_growth_rate_view_by_j
),
kpi_target as (
  select
    app_package,
    country_group,
    mediasource,
    install_day,
    kpi_target
  from lw_20250903_kpi_target_table_by_j
)
select
growth.app_package,
growth.country_group,
growth.mediasource,
growth.tag,
growth.install_day,
kpi_target.kpi_target/growth.p_r120_r7 as kpi7
from growth
left join kpi_target
on growth.app_package = kpi_target.app_package
and growth.country_group = kpi_target.country_group
and growth.mediasource = kpi_target.mediasource
and growth.install_day = kpi_target.install_day
where
growth.tag in (
  'aos_gpir_cohort_raw_avg28','aos_gpir_cohort_raw_avg56','aos_gpir_cohort_raw_avg84',
  'aos_gpir_cohort_raw_ema28','aos_gpir_cohort_raw_ema56','aos_gpir_cohort_raw_ema84',
  'ios_af_cohort_raw_avg28','ios_af_cohort_raw_avg56','ios_af_cohort_raw_avg84',
  'ios_af_cohort_raw_ema28','ios_af_cohort_raw_ema56','ios_af_cohort_raw_ema84'
)
;


In [0]:
-- 只获取最近1天的KPI
create or replace table lw_20250903_kpi_table_newest_by_j as
select
  app_package,
  country_group,
  mediasource,
  tag,
  first(kpi7) as newest_kpi7
from (
  select
    app_package,
    country_group,
    mediasource,
    tag,
    kpi7,
    row_number() over (partition by app_package, country_group, mediasource, tag order by install_day desc) as rn
  from lw_20250903_kpi_table_by_j
  where kpi7 is not null
) subquery
where rn = 1
group by app_package, country_group, mediasource, tag

In [0]:
-- 获取最小和最大kpi
create or replace table lw_20250903_kpi_table_newest_min_max_by_j as
select
  app_package,
  country_group,
  mediasource,
  min(kpi7) as min_kpi7,
  max(kpi7) as max_kpi7
from (
  select
    app_package,
    country_group,
    mediasource,
    tag,
    kpi7,
    row_number() over (partition by app_package, country_group, mediasource, tag order by install_day desc) as rn
  from lw_20250903_kpi_table_by_j
  where kpi7 is not null
) subquery
where rn = 1
group by app_package, country_group, mediasource

计算自然量,暂时只用aos_gpir_cohort_raw_ema84和20250808_20_ema84进行计算，差距不大。

In [0]:
create or replace table lw_20250903_kpi_organic_revenue_rate_view_by_j as
WITH daily_revenue AS (
  SELECT  
    app_package,  
    country_group,  
    tag,  
    install_day,
    SUM(CASE WHEN mediasource = 'Organic' THEN revenue_d120 ELSE 0 END) AS organic_revenue,
    SUM(revenue_d120) AS total_revenue
  FROM lw_20250903_real_and_predict_revenue_table_by_j  
  WHERE tag IN ('aos_gpir_cohort_raw_ema84', '20250808_20_ema84')
  GROUP BY app_package, country_group, tag, install_day
),
rolling_30day_calc AS (
  SELECT
    app_package,
    country_group,
    tag,
    install_day,
    SUM(organic_revenue) 
      OVER (
        PARTITION BY app_package, country_group, tag
        ORDER BY install_day
        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
      ) AS organic_revenue_30d,
    SUM(total_revenue) 
      OVER (
        PARTITION BY app_package, country_group, tag
        ORDER BY install_day
        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
      ) AS total_revenue_30d
  FROM daily_revenue
)
SELECT
  app_package,
  country_group,
  tag,
  install_day,
  CASE 
    WHEN total_revenue_30d > 0 
    THEN organic_revenue_30d / total_revenue_30d 
    ELSE NULL 
  END AS organic_ratio
FROM rolling_30day_calc
ORDER BY 
  app_package,
  country_group,
  tag,
  install_day DESC;